In [1]:
import pandas as pd
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
import numpy as np

#Load "Recipes.Csv"

recipes = pd.read_csv("Recipes.csv")
defects = pd.read_csv("Defects.csv")

# get columns from Recipes.csv
heat_id = list(recipes['HEAT_ID'])
material_code = list(recipes['Material_Code'])
weight = list(recipes['WEIGHT'])
# get columns from Defects.csv
heat_id_defect = list(defects['HEAT_ID'])
defect_type = list(defects['DEFECT_TYPE'])
# result
result = []
# material code unique
material_code_unique = []

# remove duplicate
unique_heat_id = list(set(heat_id))
# heat_id vs mat code
material_code_set = {}
# heat_id vs weight following mat code
weight_set = {}
# heat_id vs defect
defect_set = {}


In [2]:
for i in range(len(unique_heat_id)):
    material_code_set[unique_heat_id[i]] = []
    weight_set[unique_heat_id[i]] = []
    defect_set[unique_heat_id[i]] = []

for i in range(len(heat_id)):
    material_code_set[heat_id[i]].append(material_code[i])
    weight_set[heat_id[i]].append(weight[i])

for i in range(len(heat_id_defect)):
    try:
        defect_set[heat_id_defect[i]].append(defect_type[i])
    except:
        continue

# material unique
material_code_temp = []
defect_set_key = list(defect_set.keys())
for i in range(len(defect_set_key)):
    material_code_temp += material_code_set[defect_set_key[i]]
material_code_unique = list(set(material_code_temp))

In [3]:
# crate dataframe
defect_key_list = list(defect_set.keys())
for i in range(len(defect_key_list)):
    for j in range(len(defect_set[defect_key_list[i]])):
        temp = []
        # heat id
        temp.append(defect_key_list[i])
        # all mat
        for k in range(len(material_code_unique)):
            if material_code_unique[k] in material_code_set[defect_key_list[i]]:
                temp.append(weight_set[defect_key_list[i]][material_code_set[defect_key_list[i]].index(material_code_unique[k])])
            else:
                temp.append(0)
        # M_sum
        temp.append(sum(weight_set[defect_key_list[i]]))
        # M_Mean
        temp.append(sum(weight_set[defect_key_list[i]]) / len(weight_set[defect_key_list[i]]))
        # M_num
        temp.append(len(weight_set[defect_key_list[i]]))
        # Repeat
        temp.append(len(defect_set[defect_key_list[i]]))
        # defect id
        temp.append(defect_set[defect_key_list[i]][j])
        result.append(temp)

columns = []
columns.append('HEAT_ID')
for i in range(len(material_code_unique)):
    columns.append('Mat_' + str(material_code_unique[i]))
columns.append('M_Sum')
columns.append('M_Mean')
columns.append('M_Num')
columns.append('Repeat')
columns.append('Defect_Id')

df1 = pd.DataFrame(result, columns=columns)
df1.to_csv('test.csv')

In [4]:
relations = ['Mat_166', 'Mat_180', 'Mat_181', 'Mat_182', 'Mat_183', 'Mat_190', 'Mat_207', 'Mat_208', 'Mat_209', 'Mat_210', 'Mat_211', 'Mat_212', 'Mat_213', 'Mat_214', 'Mat_215', 'Mat_112', 'Mat_113', 'Mat_114', 'Mat_115', 'Mat_116', 'Mat_117', 'Mat_118', 'Mat_119', 'Mat_120', 'Mat_121', 'Mat_122', 'Mat_124', 'Mat_125', 'Mat_126']
defect_data = []

defect = pd.read_csv("test.csv")

heat_id = list(defect['HEAT_ID'])
for i in range(len(relations)):
    defect_data.append(list(defect[relations[i]]))
defect_id = list(defect['Defect_Id'])

In [5]:
def normalize(heatId):
    result = []
    result_y = []
    for i in range(len(heat_id)):
        if heat_id[i] == heatId:
            temp = []
            for j in range(len(defect_data)):
                temp.append(defect_data[j][i])
            result.append(temp)
            result_y.append(defect_id[i])
    return [result, result_y]

In [6]:
heatId = 1703936
mat_list = ['Mat_113--37249', 'Mat_119--1480']

def testing_data(mat_list):
    test_x = []
    mat_code = []
    mat_weight = []
    for i in range(len(mat_list)):
        mat_code.append(mat_list[i].split('--')[0])
        mat_weight.append(float(mat_list[i].split('--')[1]))
        
    for i in range(len(relations)):
        if relations[i] in mat_code:
            index = mat_code.index(relations[i])
            test_x.append(mat_weight[index])
        else:
            test_x.append(0.0)
    return test_x

In [7]:
train_x, train_y = normalize(heatId)
test_x = testing_data(mat_list)

In [8]:
clf = BinaryRelevance(
    classifier=SVC(),
    require_dense=[False, True]
)

train_x = np.array(train_x)
train_y = np.array(train_y)
train_y = np.reshape(train_y, (-1, 1))
test_x = np.array([test_x])

clf.fit(train_x, train_y)
prediction = clf.predict(test_x)

In [9]:
def get_result(pre):
    temp = []
    for i in range(len(train_y)):
        temp.append(abs(train_y[i][0] - pre[0][0]))
    
    min_value = min(temp)
    index = temp.index(min_value)
    return train_y[index][0]

In [10]:
print(get_result(prediction.A))

89
